**NGUỒN:** https://thuvienphapluat.vn

#**LẤY NỘI DUNG HỢP ĐỒNG TỪ TRANG WEB**

In [1]:
import os, re, csv, time, hashlib
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

# -------- Cấu hình ----------
BASE = "https://thuvienphapluat.vn"
LIST_URL = BASE + "/hopdong?page={page}"
NUM_PAGES = 5            # tăng nếu muốn crawl nhiều trang hơn
DELAY_SEC = 1.0
MIN_WORDS = 60           # tối thiểu số từ để giữ file
OUTPUT_DIR = "contracts_html_window"
CSV_MANIFEST = "contracts_html_window_manifest.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7",
}

# --------- Tiện ích ----------
def fetch_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    r.encoding = "utf-8"
    return r.text

def sha1_of(t: str) -> str:
    return hashlib.sha1(t.encode("utf-8", "ignore")).hexdigest()

def safe_filename(n: str) -> str:
    n = re.sub(r"[\\/*?:\"<>|]+", "_", n).strip()
    return (n or f"hopdong_{int(time.time())}.txt")[:180]

def clean_text_soft(t: str) -> str:
    if not t: return ""
    t = t.replace("\r", "")
    t = re.sub(r"[ \t]{2,}", " ", t)
    # giữ xuống dòng để bảo toàn biểu mẫu
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

# ---------- Nhận dạng hợp đồng/biểu mẫu ----------
KEY_PATTERNS = [
    r"\bHỢP\s*ĐỒNG\b", r"\bHOP\s*DONG\b",
    r"\bPHỤ\s*LỤC\b",  r"\bPHU\s*LUC\b",
    r"\bBIÊN\s*BẢN\b", r"\bBIEN\s*BAN\b",
    r"\bTHANH\s*LÝ\b", r"\bTHANH\s*LY\b",
    r"\bNGHIỆM\s*THU\b", r"\bNGHIEM\s*THU\b",
    r"\bCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\b",
    r"\bCONG HOA XA HOI CHU NGHIA VIET NAM\b",
    r"\b(PL|PL\d{1,3})[-\s]*[A-Z0-9À-Ỹ]{1,10}\b",
    r"\bMẪU\s+HỢP\s*ĐỒNG\b", r"\bMAU\s+HOP\s*DONG\b",
    r"^\s*Căn\s*cứ\s*:\s*$",   r"^\s*Can\s*cu\s*:\s*$",
    r"^\s*Số\s*:\s*.*$",       r"^\s*So\s*:\s*.*$",
    r"\bHỢP ĐỒNG TƯ VẤN XÂY DỰNG\b", r"\bHOP DONG TU VAN XAY DUNG\b",
]
KEY_RE = re.compile("|".join(KEY_PATTERNS), re.I | re.U)

def looks_like_contract(text: str) -> bool:
    return bool(text and KEY_RE.search(text))

# ---------- Các mốc "bắt đầu nội dung hợp đồng" ----------
START_ANCHORS = [
    r"\bHỢP ĐỒNG TƯ VẤN XÂY DỰNG\b",
    r"\bMẪU\s+HỢP\s*ĐỒNG\b",
    r"\bCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\b",
    r"^\s*Căn\s*cứ\s*:\s*$",
    r"^\s*Thông\s*tư\s+\d{1,3}/\d{4}/TT-[A-ZĐ]{2,}\b",
    r"^\s*Số\s*:\s*.*$",
    r"\bBIÊN\s*BẢN\b", r"\bPHỤ\s*LỤC\b",  # dự phòng
]
START_RE = re.compile("|".join(START_ANCHORS), re.I | re.U)

# ---------- Rác menu/banner chắc chắn ----------
JUNK_PATTERNS_STRICT = [
    r"^\s*THƯ VIỆN PHÁP LUẬT\s*$",
    r"^\s*Trang Thông tin điện tử tổng hợp\s*$",
    r"^\s*(Tra cứu|Văn bản và Tra cứu|Chính sách Pháp\s*luật mới|Pháp Luật|Liên hệ|Danh mục)\s*$",
    r"^\s*(Tra cứu Pháp Luật mới|Tra cứu Văn Bản trực tuyến|Tra cứu Dự thảo|Văn bản mới ban hành)\s*$",
    r"^\s*(Tra cứu Tiêu Chuẩn|ICS .*|Tra cứu Công Văn|Thuật ngữ pháp lý|Tra cứu Bản án)\s*$",
    r"^\s*(Luật sư toàn quốc|Website ngành luật|Hỏi đáp pháp luật|PHÁP LUẬT DOANH NGHIỆP)\s*$",
    r"^\s*(Bảng giá đất|Mẫu hợp đồng|Biểu mẫu|Mức phí, lệ phí|Diện tích tách thửa đất ở)\s*$",
    r"^\s*(Rss|Homepage|Giới thiệu|Hướng dẫn sử dụng|Widget|Phần mềm|Đại lý phân phối)\s*$",
    r"^\s*(Bạn hãy nhập e-mail|E-?mail\s*:).*$",
    r"^\s*(Đăng nhập|Đăng ký|Sử dụng tài khoản)\s*$",
    r"^\s*(Law\s*Net|iThong)\s*$",
    r"^\s*(Tải về|Chỉnh sửa và tải về)\s*$",
    r"^\s*Cập nhật:\s*\d{2}/\d{2}/\d{4}\s*$",
    r"^\s*\d+\.\s*$",
    r"^\s*\d+\.\s*[A-Za-zÀ-Ỹà-ỹ0-9].{0,80}$",
    r"^\s*\.\.\.|=>\s*$",
]
JUNK_RE = re.compile("|".join(JUNK_PATTERNS_STRICT), re.I | re.U)

# ---------- Lấy link chi tiết ----------
def extract_detail_links(list_html: str):
    soup = BeautifulSoup(list_html, "html.parser")
    links = []
    for a in soup.select("a[href*='/hopdong/']"):
        href = a.get("href", "")
        if "/hopdong/" in href and not href.rstrip("/").endswith("/hopdong"):
            links.append(urljoin(BASE, href))
    # bỏ link tải file trực tiếp nếu có
    links = [u for u in dict.fromkeys(links) if not re.search(r"\.(docx?|pdf)(\?|$)", u, re.I)]
    return links

# ---------- Cắt theo cửa sổ và lọc rác ----------
CONTENT_SELECTORS = [
    "div#divContent", ".hopdong-detail", ".content-detail", "article", "main", ".content", ".post-content"
]

def crop_and_clean_contract(text: str) -> str:
    if not text: return ""
    text = text.replace("\r", "")
    # không nén quá tay — giữ định dạng form
    text = re.sub(r"[ \t]{2,}", " ", text)

    lines = text.splitlines()
    # tìm chỉ số bắt đầu từ mốc
    start_idx = 0
    for i, l in enumerate(lines):
        if START_RE.search(l) or KEY_RE.search(l):
            start_idx = i
            break
    core = lines[start_idx:]  # cắt phần đầu là menu/breadcrumb

    keep = []
    for l in core:
        ls = l.strip()
        if not ls:
            keep.append(l); continue
        # luôn giữ nếu có từ khoá/mốc
        if START_RE.search(ls) or KEY_RE.search(ls):
            keep.append(l); continue
        # xoá các dòng rác chắc chắn là menu/banner
        if JUNK_RE.search(ls):
            continue
        keep.append(l)

    out = "\n".join(keep)
    out = re.sub(r"\n{3,}", "\n\n", out).strip()
    return out

def extract_html_window(url: str):
    html = fetch_html(url)
    soup = BeautifulSoup(html, "html.parser")

    node = None
    for sel in CONTENT_SELECTORS:
        node = soup.select_one(sel)
        if node: break
    raw = node.get_text("\n", strip=True) if node else soup.get_text("\n", strip=True)

    text = crop_and_clean_contract(raw)

    title_el = soup.find("h1") or soup.find("title")
    title = title_el.get_text(strip=True) if title_el else "Không tiêu đề"
    return title, text

# ---------- Ghi file ----------
def write_txt_clean(path: str, content: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write("### NOIDUNG_SACH ###\n")
        f.write(content)
        f.write("\n### END_NOIDUNG_SACH ###\n")

# ---------- Main ----------
def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    seen = set()
    total = 0

    with open(CSV_MANIFEST, "w", encoding="utf-8", newline="") as cf:
        w = csv.DictWriter(cf, fieldnames=["filename","title","url","words","sha1","crawled_at"])
        w.writeheader()

        for p in range(1, NUM_PAGES+1):
            list_url = LIST_URL.format(page=p)
            print(f"\n=== Trang {p}/{NUM_PAGES} ===")
            try:
                list_html = fetch_html(list_url)
            except Exception as e:
                print("Lỗi tải trang:", e); continue

            for link in extract_detail_links(list_html):
                print("→", link)
                try:
                    title, text = extract_html_window(link)
                except Exception as e:
                    print("HTML error:", e); continue

                if not looks_like_contract(text):
                    continue
                if len(text.split()) < MIN_WORDS:
                    continue

                digest = sha1_of(text)
                if digest in seen:
                    continue
                seen.add(digest)

                fname = safe_filename(title)
                path = os.path.join(OUTPUT_DIR, fname)
                write_txt_clean(path, text)

                w.writerow({
                    "filename": fname,
                    "title": title,
                    "url": link,
                    "words": len(text.split()),
                    "sha1": digest,
                    "crawled_at": time.strftime("%Y-%m-%d %H:%M:%S"),
                })

                total += 1
                print(f"{fname} ({len(text.split())} từ)")
                time.sleep(DELAY_SEC)

    print(f"\n=== HOÀN TẤT ===\nTổng: {total}\nLưu: {OUTPUT_DIR}\nManifest: {CSV_MANIFEST}")

if __name__ == "__main__":
    main()



=== Trang 1/5 ===
→ https://thuvienphapluat.vn/hopdong/779/HOP-DONG-TU-VAN-XAY-DUNG
HỢP ĐỒNG TƯ VẤN XÂY DỰNG (14137 từ)
→ https://thuvienphapluat.vn/hopdong/38386/MAU-HOP-DONG-TIEP-CAN-NGUON-GEN-VA-CHIA-SE-LOI-ICH-Khi-thuc-hien-chinh-quyen-02-cap
MẪU HỢP ĐỒNG TIẾP CẬN NGUỒN GEN VÀ CHIA SẺ LỢI ÍCH (Khi thực hiện chính quyền 02 cấp) (3472 từ)
→ https://thuvienphapluat.vn/hopdong/38383/HOP-DONG-HOP-TAC-TO-HOP-TAC
HỢP ĐỒNG HỢP TÁC (TỔ HỢP TÁC) (3509 từ)
→ https://thuvienphapluat.vn/hopdong/37364/BIEN-BAN-NGHIEM-THU-THANH-LY-HOP-DONG-THUC-HIEN-NHIEM-VU-SU-NGHIEP-KINH-TE-BO-XAY-DUNG
BIÊN BẢN NGHIỆM THU THANH LÝ HỢP ĐỒNG THỰC HIỆN NHIỆM VỤ SỰ NGHIỆP KINH TẾ (BỘ XÂY DỰNG) (2012 từ)
→ https://thuvienphapluat.vn/hopdong/37363/HOP-DONG-THUC-HIEN-NHIEM-VU-SU-NGHIEP-KINH-TE-BO-XAY-DUNG
HỢP ĐỒNG THỰC HIỆN NHIỆM VỤ SỰ NGHIỆP KINH TẾ (BỘ XÂY DỰNG) (2326 từ)
→ https://thuvienphapluat.vn/hopdong/37362/BIEN-BAN-THANH-LY-HOP-DONG-THUC-HIEN-NHIEM-VU-BAO-VE-MOI-TRUONG-BO-XAY-DUNG
BIÊN BẢN THANH LÝ HỢP ĐỒNG

#**TỔNG HỢP CÁC FILE HỢP ĐỒNG**

In [2]:
import os

# Thư mục chứa các file TXT
input_dir = "/content/TongHop"
output_file = "/content/noidungtong.txt"

# Mở file kết quả để ghi toàn bộ nội dung
with open(output_file, "w", encoding="utf-8") as outfile:
    # Lặp qua toàn bộ file trong thư mục
    for filename in sorted(os.listdir(input_dir)):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, "r", encoding="utf-8") as infile:
                outfile.write(f"\n\n===== {filename} =====\n\n")  # tiêu đề phân cách
                outfile.write(infile.read())
                outfile.write("\n\n")

print("Đã gộp xong tất cả các file .txt vào:", output_file)


Đã gộp xong tất cả các file .txt vào: /content/noidungtong.txt


In [3]:
from google.colab import files
files.download("/content/noidungtong.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>